In [30]:
import boto3
import json
import time
import zipfile
from io import BytesIO
import uuid
import pprint
import logging


In [31]:

logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

[2025-08-14 21:19:40,556] p33998 {credentials.py:1356} INFO - Found credentials in shared credentials file: ~/.aws/credentials


In [32]:
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region

'us-east-1'

In [33]:
inference_profile = "us.amazon.nova-micro-v1:0" 
foundation_model = inference_profile[3:]
foundation_model

'amazon.nova-micro-v1:0'

In [34]:
suffix = f"{region}-{account_id}"
agent_name = "hr-assistant-function-def"
agent_bedrock_allow_policy_name = f"{agent_name}-ba-{suffix}"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'
agent_description = "Agent for providing HR assistance to manage vacation time"
agent_instruction = "You are an HR agent, helping employees understand HR policies and manage vacation time"
agent_action_group_name = "VacationsActionGroup"
agent_action_group_description = "Actions for getting the number of available vacations days for an employee and confirm new time off"
agent_alias_name = f"{agent_name}-alias"
lambda_function_role = f'{agent_name}-lambda-role-{suffix}'
lambda_function_name = f'{agent_name}-{suffix}'

In [35]:

agent_functions = [
    {
        'name': 'get_available_vacations_days',
        'description': 'get the number of vacations available for a certain employee',
        'parameters': {
            "employee_id": {
                "description": "the id of the employee to get the available vacations",
                "required": True,
                "type": "integer"
            }
        }
    },
    {
        'name': 'reserve_vacation_time',
        'description': 'reserve vacation time for a specific employee - you need all parameters to reserve vacation time',
        'parameters': {
            "employee_id": {
                "description": "the id of the employee for which time off will be reserved",
                "required": True,
                "type": "integer"
            },
            "start_date": {
                "description": "the start date for the vacation time",
                "required": True,
                "type": "string"
            },
            "end_date": {
                "description": "the end date for the vacation time",
                "required": True,
                "type": "string"
            }
        }
    },
]

In [36]:
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId="CDL0KSVUWE",
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': 'arn:aws:lambda:us-east-1:447924746755:function:hr-assistant-function-def-us-east-1-447924746755'
    },
    actionGroupName=agent_action_group_name,
    functionSchema={
        'functions': agent_functions
    },
    description=agent_action_group_description
)

agent_action_group_response

ConflictException: An error occurred (ConflictException) when calling the CreateAgentActionGroup operation: Could not perform Create operation, since the VacationsActionGroup (id: YTNOJR6BN1) with the same name VacationsActionGroup already exists

In [29]:
response = lambda_client.add_permission(
    FunctionName=lambda_function_name,
    StatementId='allow_bedrock',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com',
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/CDL0KSVUWE",
)

ResourceConflictException: An error occurred (ResourceConflictException) when calling the AddPermission operation: The statement id (allow_bedrock) provided already exists. Please provide a new statement id, or remove the existing statement.

In [23]:
response = bedrock_agent_client.prepare_agent(
    agentId="CDL0KSVUWE"
)
response

{'ResponseMetadata': {'RequestId': 'f9b9be89-9100-433c-a552-ec9738333a3c',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Fri, 15 Aug 2025 05:47:56 GMT',
   'content-type': 'application/json',
   'content-length': '119',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'f9b9be89-9100-433c-a552-ec9738333a3c',
   'x-amz-apigw-id': 'PVSB8GVBIAMEOwA=',
   'x-amzn-trace-id': 'Root=1-689eca0c-3fd15efb5e335cc839971cc5'},
  'RetryAttempts': 0},
 'agentId': 'CDL0KSVUWE',
 'agentStatus': 'PREPARING',
 'agentVersion': 'DRAFT',
 'preparedAt': datetime.datetime(2025, 8, 15, 5, 47, 56, 589650, tzinfo=tzutc())}

In [25]:
response = bedrock_agent_client.create_agent_alias(
    agentAliasName='test-alias-1',
    agentId="CDL0KSVUWE"
)

response

{'ResponseMetadata': {'RequestId': '8473ebf6-81d9-4833-b924-ef6859d8206f',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Fri, 15 Aug 2025 05:49:56 GMT',
   'content-type': 'application/json',
   'content-length': '382',
   'connection': 'keep-alive',
   'x-amzn-requestid': '8473ebf6-81d9-4833-b924-ef6859d8206f',
   'x-amz-apigw-id': 'PVSUvEG8oAMEdoQ=',
   'x-amzn-trace-id': 'Root=1-689eca84-192a88a167576d6e2f84ef3d'},
  'RetryAttempts': 0},
 'agentAlias': {'agentAliasArn': 'arn:aws:bedrock:us-east-1:447924746755:agent-alias/CDL0KSVUWE/SY5ZUHIIQ2',
  'agentAliasId': 'SY5ZUHIIQ2',
  'agentAliasName': 'test-alias-1',
  'agentAliasStatus': 'CREATING',
  'agentId': 'CDL0KSVUWE',
  'aliasInvocationState': 'ACCEPT_INVOCATIONS',
  'createdAt': datetime.datetime(2025, 8, 15, 5, 49, 56, 502183, tzinfo=tzutc()),
  'routingConfiguration': [{}],
  'updatedAt': datetime.datetime(2025, 8, 15, 5, 49, 56, 502183, tzinfo=tzutc())}}

In [38]:
agent_action_group_response = bedrock_agent_client.update_agent_action_group(
      agentId="CDL0KSVUWE",
      agentVersion='DRAFT',
      actionGroupId='YTNOJR6BN1',  # Use the existing action group ID
      actionGroupExecutor={
          'lambda': 'arn:aws:lambda:us-east-1:447924746755:function:hr-assistant-function-def-us-east-1-447924746755'
      },
      actionGroupName=agent_action_group_name,
      functionSchema={
          'functions': agent_functions
      },
      description=agent_action_group_description
  )

agent_action_group_response

{'ResponseMetadata': {'RequestId': '8419b773-063d-44b4-9303-10f3c70e7393',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 15 Aug 2025 06:22:56 GMT',
   'content-type': 'application/json',
   'content-length': '1388',
   'connection': 'keep-alive',
   'x-amzn-requestid': '8419b773-063d-44b4-9303-10f3c70e7393',
   'x-amz-apigw-id': 'PVXKGEZlIAMEsVw=',
   'x-amzn-trace-id': 'Root=1-689ed240-76a7801e5e479b5173a7ea16'},
  'RetryAttempts': 0},
 'agentActionGroup': {'actionGroupExecutor': {'lambda': 'arn:aws:lambda:us-east-1:447924746755:function:hr-assistant-function-def-us-east-1-447924746755'},
  'actionGroupId': 'YTNOJR6BN1',
  'actionGroupName': 'VacationsActionGroup',
  'actionGroupState': 'ENABLED',
  'agentId': 'CDL0KSVUWE',
  'agentVersion': 'DRAFT',
  'clientToken': 'd56516b7-7da5-4a8f-ad19-9e590fe5a522',
  'createdAt': datetime.datetime(2025, 8, 15, 5, 44, 45, 252606, tzinfo=tzutc()),
  'description': 'Actions for getting the number of available vacations days for an empl

In [41]:
prepare_agent_response = bedrock_agent_client.prepare_agent(
      agentId="CDL0KSVUWE"
  )

prepare_agent_response

{'ResponseMetadata': {'RequestId': 'f48fce5b-8d21-4a5c-b60e-07d9325e11b4',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Fri, 15 Aug 2025 06:24:02 GMT',
   'content-type': 'application/json',
   'content-length': '119',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'f48fce5b-8d21-4a5c-b60e-07d9325e11b4',
   'x-amz-apigw-id': 'PVXUbHy8oAMEpFA=',
   'x-amzn-trace-id': 'Root=1-689ed282-0c2866d54e3dac53617bc0d4'},
  'RetryAttempts': 0},
 'agentId': 'CDL0KSVUWE',
 'agentStatus': 'PREPARING',
 'agentVersion': 'DRAFT',
 'preparedAt': datetime.datetime(2025, 8, 15, 6, 24, 2, 747308, tzinfo=tzutc())}

In [42]:
prepare_agent_response

{'ResponseMetadata': {'RequestId': 'f48fce5b-8d21-4a5c-b60e-07d9325e11b4',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Fri, 15 Aug 2025 06:24:02 GMT',
   'content-type': 'application/json',
   'content-length': '119',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'f48fce5b-8d21-4a5c-b60e-07d9325e11b4',
   'x-amz-apigw-id': 'PVXUbHy8oAMEpFA=',
   'x-amzn-trace-id': 'Root=1-689ed282-0c2866d54e3dac53617bc0d4'},
  'RetryAttempts': 0},
 'agentId': 'CDL0KSVUWE',
 'agentStatus': 'PREPARING',
 'agentVersion': 'DRAFT',
 'preparedAt': datetime.datetime(2025, 8, 15, 6, 24, 2, 747308, tzinfo=tzutc())}